In [1]:
!pip install langchain langchain_community langchain_chroma -qU langchain-openai bs4 googlemaps

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import os

# Python 코드에서 환경 변수 설정
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "langchain"
os.environ["GOOGLE_API_KEY"] = "google"
os.environ["OPENAI_API_KEY"] = "openai"

import googlemaps
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from datetime import datetime
from langchain.schema import Document
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")

In [54]:
# Google Maps API 클라이언트 초기화
api_key = ''
gmaps = googlemaps.Client(key=api_key)

def get_place_reviews(city_name, max_places=10):
    # 도시의 좌표를 찾기 위해 Geocoding API를 사용
    geocode_result = gmaps.geocode(city_name)
    if not geocode_result:
        return []

    # 도시의 중앙 좌표를 추출
    city_location = geocode_result[0]['geometry']['location']

    # Places API를 사용하여 관광지 검색
    places_result = gmaps.places_nearby(
        location=(city_location['lat'], city_location['lng']),
        radius=10000,  # 10km 반경 내의 장소 검색
        type='tourist_attraction'
    )

    docs = []
    unique_docs = []

    # 관광지에서 리뷰를 가져오기
    for place in places_result.get('results', [])[:max_places]:
        place_id = place['place_id']
        place_details = gmaps.place(place_id=place_id, fields=['name', 'reviews'])

        place_name = place_details.get('result', {}).get('name', 'No name')
        reviews = place_details.get('result', {}).get('reviews', [])

        # 리뷰를 docs 리스트에 추가
        if reviews:
            review_texts = []
            review_texts = [f"Review for {place_name}:\n\n{review['text']}" for review in reviews]
            docs.extend(review_texts)
        else:
            docs.append(f"No reviews available for {place_name}.")

    return docs

# 예시 사용
#docs = get_place_reviews(city_name)

# docs 출력
#for doc in docs:
 #   print(doc)
  #  print("\n---\n")


In [66]:
from functools import lru_cache

@lru_cache(maxsize=None)
def some_function(x):
    return x * x

# 캐시를 비우기
some_function.cache_clear()


In [ ]:
city_name = input("도시를 입력하세요 (예: 타이페이): ")
user_request = input("여행 관심사를 입력하세요 (예: 역사적인 관광지): ")

all_review = []
docs = []
doc = []
unique_docs = []
all_review = get_place_reviews(city_name)

#all_review
#docs

In [68]:

# 문자열을 문서 객체로 변환
docs = [Document(page_content=text) for text in all_review]  # 'texts'는 문서 내용이 담긴 문자열 리스트

# 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# 벡터 저장소 생성
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# 검색 및 생성
retriever = vectorstore.as_retriever(search_kwargs={"k": 50})

# Prompt
template = '''없는 정보는 자체 제공하여도 되나 최대한
제공되는 리뷰에 맞추어 대답해주세요:
{context}

Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)


# Rretriever
retriever = vectorstore.as_retriever()

# Combine Documents
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

# RAG Chain 연결
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Chain 실행
rag_chain.invoke(f"""도시"{city_name}"의"{user_request}"의 관한 여행계획 추천해주세요""")

'서울의 역사적인 관광지로는 광화문 광장, 경복궁, 덕수궁, 청계천, 서울 시청 등이 있습니다. 이 중에서 광화문 광장에 위치한 세종대왕과 이순신 장군의 동상, 그리고 근처에 있는 경복궁과 덕수궁은 꼭 방문해보시는 것을 추천드립니다. 또한 청계천과 서울 시청도 역사적인 가치가 높은 관광지이니 함께 방문해보시면 좋을 것입니다. 광화문 광장에서 시작하여 이 근처의 다양한 역사적인 장소를 돌아보는 여행을 계획해보세요.'

LangChain 이용 RAG 사용
Retriever 초기화 안됨 수정필요

수정용 테스트 코드

In [ ]:

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def remove_duplicates(docs):
    seen = set()
    unique_docs = []
    for doc in docs:
        if doc.page_content not in seen:
            unique_docs.append(doc)
            seen.add(doc.page_content)
    return unique_docs

# 중복 제거 후 문서 분할
unique_docs = remove_duplicates(docs)
splits = text_splitter.split_documents(unique_docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [17]:

# 문자열을 문서 객체로 변환
docs = [Document(page_content=text) for text in all_review]  # 'texts'는 문서 내용이 담긴 문자열 리스트

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def remove_duplicates(docs):
    seen = set()
    unique_docs = []
    for doc in docs:
        if doc.page_content not in seen:
            unique_docs.append(doc)
            seen.add(doc.page_content)
    return unique_docs

# 중복 제거 후 문서 분할
unique_docs = remove_duplicates(docs)
splits = text_splitter.split_documents(unique_docs)

# 벡터 저장소 생성
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# 검색 및 생성
retriever = vectorstore.as_retriever(search_kwargs={"k": len(docs)})

# Prompt
template = '''제공되는 리뷰에 맞추어 대답해주세요:
{context}

Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)


# Rretriever
retriever = vectorstore.as_retriever()

# Combine Documents
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

# RAG Chain 연결
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Chain 실행
rag_chain.invoke(f"""도시"{city_name}"의"{user_request}"의 관한 여행계획 추천해주세요""")

"I'm sorry, I can only provide information and answer questions related to attractions in Seoul. Let me know if you need any recommendations for places to visit in Seoul!"

In [30]:

# 문자열을 문서 객체로 변환
docs = [Document(page_content=text) for text in all_review]  # 'texts'는 문서 내용이 담긴 문자열 리스트

# 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# 벡터 저장소 생성
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# 검색 및 생성
retriever = vectorstore.as_retriever(search_kwargs={"k": 50})

# Prompt 설정
template = '''없는 정보는 자체 제공하여도 되나 최대한
제공되는 리뷰에 맞추어 대답해주세요:
{context}

Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)
llm = ChatOpenAI(model='gpt-3.5-turbo-0125', temperature=0)

# 문서 포맷팅 함수
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs if doc.page_content)

# Context를 동적으로 생성하는 함수
def generate_context(query):
    # 검색 수행 (get_relevant_documents 메서드 사용)
    search_results = retriever.get_relevant_documents(query)
    return format_docs(search_results)

# RAG Chain 설정 함수
def create_rag_chain(retriever, prompt, model):
    def context_runnable(query):
        # 새 쿼리에 대해 새로운 컨텍스트 생성
        return generate_context(query)

    # 데이터 흐름 설정
    return (
        {'context': RunnablePassthrough(func=context_runnable), 'question': RunnablePassthrough()}
        | prompt
        | model
        | StrOutputParser()
    )

# RAG Chain 생성
rag_chain = create_rag_chain(retriever, prompt, llm)

# Chain 실행
query = f'도시 "{city_name}"의 "{user_request}"에 관한 여행계획 추천해주세요'
result = rag_chain.invoke({'context': query, 'question': query})
print(result)


<ipython-input-30-0943e6ef3815>:32: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  search_results = retriever.get_relevant_documents(query)


TypeError: argument 'text': 'dict' object cannot be converted to 'PyString'

In [12]:


# 문서 리스트와 텍스트 분할기 설정
docs = [Document(page_content=text) for text in all_review]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# 벡터 저장소 생성
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# retriever 초기화
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# Prompt 템플릿 정의
template = '''없는 정보는 자체 제공하여도 되나 최대한
제공되는 리뷰에 맞추어 대답해주세요:
{context}

Question: {question}
'''
prompt = ChatPromptTemplate.from_template(template)

# Combine Documents 함수 정의
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

# RAG Chain 연결
def create_rag_chain():
    return (
        {'context': retriever | format_docs, 'question': RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

# RAG Chain 실행 함수 정의
def run_rag_chain(query, city_name, user_request):
    # query를 포함한 문서 검색
    search_results = retriever.get_relevant_documents(query)

    # 검색된 문서들을 포맷팅
    context = format_docs(search_results)

    # RAG Chain 생성
    rag_chain = create_rag_chain()

    # Chain 실행
    return rag_chain.invoke(f"""도시 "{city_name}"의 "{user_request}"에 관한 여행 계획 추천해주세요""")

# 예시 실행
result = run_rag_chain(f"도시 '{city_name}'의 '{user_request}'에 관한 여행 계획 추천해주세요", city_name, user_request)

print(result)


다낭의 역사적인 관광지로는 반다망 요새와 미소다이 다성이 있습니다. 반다망 요새는 프랑스 식민지 시대의 유적으로, 다낭의 역사와 문화를 경험할 수 있는 곳입니다. 미소다이 다성은 19세기 후반에 지어진 요새로서, 다낭의 전쟁 시기 역사를 엿볼 수 있는 장소입니다. 이 두 곳을 방문하면 다낭의 역사적인 면을 보다 깊이 이해할 수 있을 것입니다. 또한 현지 가이드를 통해 자세한 역사적 배경을 알아보는 것도 좋은 방법입니다.


In [11]:
# 문자열을 문서 객체로 변환
docs = [Document(page_content=text) for text in all_review]

# 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# 벡터 저장소 생성
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# retriever 초기화
retriever = vectorstore.as_retriever(search_kwargs={"k": 50})

# Prompt 템플릿 정의
template = '''없는 정보는 자체 제공하여도 되나 최대한
제공되는 리뷰에 맞추어 대답해주세요:
{context}

Question: {question}
'''
prompt = ChatPromptTemplate.from_template(template)

# Combine Documents 함수 정의
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

# RAG Chain 생성 함수 정의
def create_rag_chain():
    return (
        {'context': retriever | format_docs, 'question': RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

# RAG Chain 실행 함수 정의
def run_rag_chain(query, city_name, user_request):
    # query를 포함한 문서 검색
    search_results = retriever.get_relevant_documents(query)

    # 검색된 문서들을 포맷팅
    context = format_docs(search_results)

    # RAG Chain 생성
    rag_chain = create_rag_chain()

    # Chain 실행
    return rag_chain.invoke(f"""도시 "{city_name}"의 "{user_request}"에 관한 여행 계획 추천해주세요""")

# 예시 실행
result = run_rag_chain(f"도시 '{city_name}'의 '{user_request}'에 관한 여행 계획 추천해주세요", city_name, user_request)

print(result)

- 다낭의 역사적인 관광지 중 하나로 "숭어 다리"를 방문해보세요. 이 곳은 다낭의 아름다운 전통 다리 중 하나로 역사적인 중요성을 가지고 있습니다. 숭어 다리를 건너면서 도시의 역사와 문화에 대해 더 많이 알아갈 수 있습니다.
- 또한, "다낭 역사 박물관"을 방문하여 도시의 과거와 현재에 대한 흥미로운 정보를 얻을 수 있습니다. 박물관은 최근에 리모델링되어 전시물들이 새롭게 정리되었습니다. 다양한 아티팩트들을 통해 다낭의 다채로운 역사와 문화를 경험해보세요.


In [10]:
# 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# 벡터 저장소 생성
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# retriever 초기화
retriever = vectorstore.as_retriever(search_kwargs={"k": 50})

# Prompt 템플릿 정의
template = '''없는 정보는 자체 제공하여도 되나 최대한
제공되는 리뷰에 맞추어 대답해주세요:
{context}

Question: {question}
'''
prompt = ChatPromptTemplate.from_template(template)

# Combine Documents 함수 정의
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

# RAG Chain 생성 함수 정의
def create_rag_chain():
    return (
        {'context': retriever | format_docs, 'question': RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

# RAG Chain 실행 함수 정의
def run_rag_chain(query, city_name, user_request):
    # query를 포함한 문서 검색
    search_results = retriever.get_relevant_documents(query)

    # 검색된 문서들을 포맷팅
    context = format_docs(search_results)

    # RAG Chain 생성
    rag_chain = create_rag_chain()

    # Chain 실행
    return rag_chain.invoke(f"""도시 "{city_name}"의 "{user_request}"에 관한 여행 계획 추천해주세요""")

# 예시 실행
result = run_rag_chain(f"도시 '{city_name}'의 '{user_request}'에 관한 여행 계획 추천해주세요", city_name, user_request)

print(result)

ValueError: Expected IDs to be a non-empty list, got 0 IDs